# 주차장 프로젝트 (Colab용)

현재 폴더 구조에 맞춘 자동 라벨 병합 + 학습 노트북입니다.

## 포함 기능
- (옵션) `dataset_car.zip` 업로드 후 자동 해제
- `work_auto/predict` 자동 라벨 후처리→`train` 병합
- `data.yaml` 생성
- YOLOv8 학습(증강 포함) / (선택) 파인튜닝 / ONNX export

※ Colab 경로 기준 `/content/dataset_car` 를 기본으로 사용합니다.

In [ ]:
# 런타임 준비(Colab)
!pip -q install ultralytics opencv-python-headless shapely
from ultralytics import YOLO
import os, glob, shutil, random, cv2, numpy as np
from pathlib import Path
print('Ultralytics version:', YOLO.__module__.split('.')[0])
print('OpenCV version:', cv2.__version__)

# 경로 설정
BASE = Path('/content/dataset_car')
BASE.mkdir(parents=True, exist_ok=True)
print('BASE =', BASE.resolve())

## (옵션) dataset_car.zip 업로드 후 자동 해제
- 로컬의 `dataset_car`를 zip으로 묶어 업로드하면 자동으로 `/content/dataset_car`로 풀립니다.
- 이미 런타임에 `/content/dataset_car`가 있다면 이 셀은 건너뛰어도 됩니다.

In [ ]:
from google.colab import files
if not any(BASE.iterdir()):
  print('dataset_car.zip 을 업로드하세요 (선택).')
  uploaded = files.upload()
  for fn in uploaded:
    if fn.endswith('.zip'):
      import zipfile
      with zipfile.ZipFile(fn, 'r') as z:
        z.extractall('/content')
      print('Unzipped to /content/')
      break
print('구성 완료(없으면 기존 폴더 사용).')

## 폴더 구조 점검
- 기대하는 구조 (사용자님 현황에 맞춤)
```
dataset_car/
 ├─ images/           # 수동 라벨 원본 이미지 (있을 수도/없을 수도)
 ├─ labels/           # 수동 라벨 txt (빈 txt 포함 가능)
 ├─ raw_auto/         # 자동 라벨링 대상 원본 이미지
 ├─ train/ images,labels   # 이미 일부 구성되어 있을 수 있음
 ├─ val/   images,labels   # 이미 일부 구성되어 있을 수 있음
 └─ work_auto/predict/(labels, images)
```

In [ ]:
IMG_EXTS = ('.jpg', '.jpeg', '.png', '.bmp')
def count_dir(p: Path):
  return sum(1 for _ in p.glob('*')) if p.exists() else 0

paths = {
  'images': BASE/'images',
  'labels': BASE/'labels',
  'raw_auto': BASE/'raw_auto',
  'train_images': BASE/'train/images',
  'train_labels': BASE/'train/labels',
  'val_images': BASE/'val/images',
  'val_labels': BASE/'val/labels',
  'wa_pred': BASE/'work_auto/predict',
  'wa_labels': BASE/'work_auto/predict/labels',
}
for k, p in paths.items():
  print(f"{k:13s}", p, '| exists:', p.exists(), '| entries:', count_dir(p))

# train/val 폴더 보장
paths['train_images'].mkdir(parents=True, exist_ok=True)
paths['train_labels'].mkdir(parents=True, exist_ok=True)
paths['val_images'].mkdir(parents=True, exist_ok=True)
paths['val_labels'].mkdir(parents=True, exist_ok=True)
print('폴더 점검 완료')

## 자동 라벨 후처리 + train 병합
- `work_auto/predict/labels/*.txt` 기준으로 **해당 이미지**를 찾아 후처리
- 이미지 위치는 `work_auto/predict/` 또는 `work_auto/predict/images/` 모두 탐색
- 필터: 너무 작은 박스/비정상 종횡비 제거, (옵션) ROI 내부만 유지
- COCO car/bus/truck → `car(0)` 통일
- 탐지 없음 이미지도 **빈 .txt 생성** 후 train에 포함 (네거티브 유지)
- **val 세트는 건드리지 않음** (수동 라벨만 유지 권장)

In [ ]:
MIN_W, MIN_H = 0.03, 0.03
AR_MIN, AR_MAX = 0.4, 3.0
ROI_POLY = []  # 예: [(100,100),(1180,100),(1180,620),(100,620)]  참조 프레임 좌표. 없으면 빈 리스트 유지

def point_in_poly(x, y, poly):
  if not poly: return True
  poly_np = np.array(poly, dtype=np.int32).reshape(-1,1,2)
  return cv2.pointPolygonTest(poly_np, (float(x),float(y)), False) >= 0

def yolo_empty(txt_path: Path):
  txt_path.parent.mkdir(parents=True, exist_ok=True)
  if not txt_path.exists():
    open(txt_path, 'w').close()

pred_dir = paths['wa_pred']
pred_lab = paths['wa_labels']
if pred_lab.exists():
  # 후보 이미지 폴더: predict/, predict/images/
  cand_dirs = [pred_dir, pred_dir/'images']
  img_map = {}
  for d in cand_dirs:
    if d.exists():
      for ext in IMG_EXTS:
        for ip in d.glob(f'*{ext}'):
          img_map[ip.stem] = ip

  removed, kept, empty_made = 0, 0, 0
  for txt in pred_lab.glob('*.txt'):
    stem = txt.stem
    ip = img_map.get(stem, None)
    if ip is None:
      # 예측 이미지가 없으면 스킵(라벨만 있는 특수 케이스)
      continue
    img = cv2.imread(str(ip))
    if img is None:
      continue
    H, W = img.shape[:2]
    out_lines = []
    with open(txt) as f:
      for line in f:
        parts = line.strip().split()
        if len(parts) < 5: continue
        # parts[0]=원래 클래스(2/5/7 등). 저장은 항상 car=0
        cx, cy, w, h = map(float, parts[1:5])
        if w < MIN_W or h < MIN_H: 
          removed += 1; continue
        ar = w/h if h>1e-6 else 999.0
        if not (AR_MIN <= ar <= AR_MAX):
          removed += 1; continue
        px, py = cx*W, cy*H
        if not point_in_poly(px, py, ROI_POLY):
          removed += 1; continue
        out_lines.append(f"0 {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}")

    # 후처리 결과 저장(덮어쓰기)
    with open(txt, 'w') as f:
      f.write('\n'.join(out_lines))
    kept += len(out_lines)

  # 탐지 결과가 전혀 없어 .txt 자체가 없는 이미지도 빈 txt 생성
  for stem, ip in img_map.items():
    lp = pred_lab/f"{stem}.txt"
    if not lp.exists():
      yolo_empty(lp)
      empty_made += 1

  print(f"[INFO] 후처리: kept boxes={kept}, removed={removed}, empty_txt_created={empty_made}")

  # train에 병합 (val은 유지)
  added = 0
  for stem, ip in img_map.items():
    lp = pred_lab/f"{stem}.txt"
    if not lp.exists():
      continue
    shutil.copy(ip, paths['train_images']/ip.name)
    shutil.copy(lp, paths['train_labels']/(stem + '.txt'))
    added += 1
  print(f"[INFO] 자동 라벨 병합: train에 {added}장 추가")
else:
  print('[WARN] work_auto/predict/labels 경로가 없습니다. 자동 라벨 병합 생략.')

`data.yaml` 생성/갱신 (1클래스 `car` 고정)

In [ ]:
yaml_text = f"""
path: {BASE}
train: train/images
val: val/images
nc: 1
names: [car]
"""
with open(BASE/'data.yaml', 'w') as f:
  f.write(yaml_text)
print((BASE/'data.yaml').read_text())

## YOLOv8 학습 (증강 포함)
- ROI 기반 점유 판정에 맞춰 **강한 기하변형은 제한**, 조명/색/약한 기하 위주 증강
- 작은 데이터셋에서 효과적인 `mosaic/mixup` 사용(과대하면 불안정해서 낮은 값)

In [ ]:
model = YOLO('yolov8n.pt')  # 속도 우선. 정확도 필요 시 yolov8s.pt
model.train(
    data=str(BASE/'data.yaml'),
    epochs=80,
    batch=32,
    imgsz=640,
    lr0=0.001,
    patience=15,
    device=0,
    name='car_mix_aug_colab',
    # 증강 설정
    hsv_h=0.015, hsv_s=0.70, hsv_v=0.40,
    fliplr=0.20, flipud=0.0,
    scale=0.10, translate=0.02,
    degrees=3.0, shear=2.0, perspective=0.0005,
    mosaic=0.70, mixup=0.10, copy_paste=0.0
)
print('[INFO] 1차 학습 완료')

## (선택) 파인튜닝 10 epoch
- 마지막 구간에서 `mosaic/mixup`을 끄고 **실제 분포 적응**
- 필요 없으면 건너뛰세요.

In [ ]:
best = '/content/runs/detect/car_mix_aug_colab/weights/best.pt'
if Path(best).exists():
  YOLO(best).train(
      data=str(BASE/'data.yaml'),
      epochs=10, batch=32, imgsz=640, lr0=5e-4,
      patience=5, device=0, name='car_mix_aug_colab_ft',
      mosaic=0.0, mixup=0.0
  )
  print('[INFO] 파인튜닝 완료')
else:
  print('[WARN] best.pt 미존재: 파인튜닝 생략')

## (선택) ONNX 내보내기 (TensorRT 변환용)
- 젯슨 나노에서 TensorRT 변환 예정이면 실행하세요.

In [ ]:
final_pt = '/content/runs/detect/car_mix_aug_colab_ft/weights/best.pt'
if not Path(final_pt).exists():
  final_pt = '/content/runs/detect/car_mix_aug_colab/weights/best.pt'
if Path(final_pt).exists():
  print('Exporting ONNX from:', final_pt)
  YOLO(final_pt).export(format='onnx', opset=12, dynamic=True)
  print('[INFO] ONNX export done')
else:
  print('[WARN] best.pt 없음: ONNX export 생략')